# **Key Point Detection -1**

### **Image Augmentation**

Images are augmented and its coresponding annotation is created using this script

For this I am using Dataset from Roboflow Universe
[Dataset Link](https://public.roboflow.com/object-detection/chess-full)

The edges of the chess board were labeled using Labelme desktop software, as the Roboflow platform did not support keypoint annotation at that time.



```
# If you run this on your dataset and your colab account only you need to do is change the file paths
```



In [ ]:
%cd /content/drive/MyDrive/KeyPoint

/content/drive/MyDrive/KeyPoint


In [ ]:
!ls

Chess_Augmentation.ipynb  data	data_creation.ipynb


In [ ]:
import os
import albumentations as alb
import cv2
import json
import numpy as np

### **Augmentations**
1. Random Brightness Contrast
2. Random Gamma
3. RGB Shift

In [ ]:
augmentor = alb.Compose([alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2)], 
                         keypoint_params=alb.KeypointParams(format='xy', label_fields=['class_labels']))

Create a new folder and save augmented data for it

In [ ]:
data_dir = r'/content/drive/MyDrive/KeyPoint/data'
save_dir = r'/content/drive/MyDrive/KeyPoint/data/aug3'

save_img_dir = os.path.join(save_dir,'images')
save_label_dir = os.path.join(save_dir,'labels')

if not os.path.exists(save_img_dir):
  os.makedirs(save_img_dir)
if not os.path.exists(save_label_dir):
  os.makedirs(save_label_dir)

In [ ]:
from pathlib import Path

In [ ]:
# Labels --> LU RU LD RD
# LU 0 1
# RU 2 3
# LD 4 5
# RD 6 7

class_labels = ['LU', 'RU' ,'LD' ,'RD']

In [ ]:
for image in os.listdir(os.path.join(data_dir, 'images')):
    img = cv2.imread(os.path.join(data_dir, 'images', image))

    classes = [0] * 4
    coords = [0] * 8
    image_name = Path(image).stem

    label_path = os.path.join(data_dir, 'labels', f'{image_name}.json')
    print(label_path)

    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            label = json.load(f)

        h = label["imageHeight"]
        w = label["imageWidth"]
        #print(h,w)

        for shape in label['shapes']:
          if shape['label']== 'LU': 
            classes[0] = 1
            coords[0] = np.squeeze(shape['points'])[0]  # x
            coords[1] = np.squeeze(shape['points'])[1]  # y

          elif shape['label']== 'RU': 
            classes[1] = 1
            coords[2] = np.squeeze(shape['points'])[0]
            coords[3] = np.squeeze(shape['points'])[1]
        
          elif shape['label']== 'LD': 
            classes[2] = 1
            coords[4] = np.squeeze(shape['points'])[0]
            coords[5] = np.squeeze(shape['points'])[1]

          elif shape['label']== 'RD': 
            classes[3] = 1
            coords[6] = np.squeeze(shape['points'])[0]
            coords[7] = np.squeeze(shape['points'])[1]

        print(coords)
        # normalize
        d_vec = [w,h, w,h, w,h, w,h]
        np.divide(coords, d_vec)

    try: 
      for x in range(5):
          keypoints = [(coords[:2]), (coords[:4]) , (coords[:6]), (coords[:8])]
          augmented = augmentor(image=img, keypoints=keypoints, class_labels=['LU', 'RU', 'LD', 'RD'])
          #print(augmented)
          
          cv2.imwrite(os.path.join(save_img_dir, f'{image_name.split(".")[0]}_{x}.jpg'), augmented['image'])

          annotation = {}
          annotation['image'] = image
          annotation['class'] = [0] * 4
          annotation['keypoints'] = [0] * 8

          if os.path.exists(label_path):
              if len(augmented['keypoints']) > 0: 
                  for idx, cl in enumerate(augmented['class_labels']):
                    #print(cl)
                    if cl == 'LU': 
                        annotation['class'][0] = 1 
                        print(annotation['keypoints'])
                        annotation['keypoints'][0] = augmented['keypoints'][idx][0]
                        annotation['keypoints'][1] = augmented['keypoints'][idx][1]
                    
                    elif cl == 'RU': 
                        annotation['class'][1] = 1 
                        annotation['keypoints'][2] = augmented['keypoints'][idx][2]
                        annotation['keypoints'][3] = augmented['keypoints'][idx][3]
                    
                    elif cl == 'LD': 
                        annotation['class'][2] = 1 
                        annotation['keypoints'][4] = augmented['keypoints'][idx][4]
                        annotation['keypoints'][5] = augmented['keypoints'][idx][5]

                    elif cl == 'RD': 
                        annotation['class'][3] = 1 
                        annotation['keypoints'][6] = augmented['keypoints'][idx][6]
                        annotation['keypoints'][7] = augmented['keypoints'][idx][7]
                          
          annotation['keypoints'] = list(np.divide(annotation['keypoints'], d_vec))


          with open(os.path.join(save_label_dir, f'{image_name.split(".")[0]}_{x}.json'), 'w') as f:
              json.dump(annotation, f)

    except Exception as e:
      print(e)


/content/drive/MyDrive/KeyPoint/data/labels/7e97f49e613a59a70b833e4c0b2c1c04_jpg.rf.a1e88ea551f203aa622fb4b959241ad3.json
[65.70925110132161, 31.348017621145384, 313.7268722466961, 34.872246696035255, 31.348017621145388, 373.63876651982383, 375.40088105726875, 363.9471365638767]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
/content/drive/MyDrive/KeyPoint/data/labels/8ec14357f5f18fb98db86e0283623150_jpg.rf.bd219e41916cc083a49cc36b41f392f7.json
[65.70925110132161, 30.466960352422916, 314.6079295154185, 33.99118942731279, 31.78854625550663, 371.87665198237886, 374.51982378854626, 363.50660792951544]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
/content/drive/MyDrive/KeyPoint/data/labels/7e862b85e33cd247ed66447d129e5fb4_jpg.rf.d724a7b61892f010e784e1fb02163fde.json
[65.26872246696037, 31.348017621145384, 314.6079295154185, 33.9911894

In [ ]:
# for image in os.listdir(os.path.join('data', 'images')):
#     img = cv2.imread(os.path.join('data', 'images', image))

#     classes = [0,0]
#     coords = [0,0,0.00001,0.00001]
#     label_path = os.path.join('data', 'labels', f'{image.split(".")[0]}.json')
#     if os.path.exists(label_path):
#         with open(label_path, 'r') as f:
#             label = json.load(f)

#         if label['shapes'][0]['label']== 'LeftEye': 
#             classes[0] = 1
#             coords[0] = np.squeeze(label['shapes'][0]['points'])[0]
#             coords[1] = np.squeeze(label['shapes'][0]['points'])[1]

#         if label['shapes'][0]['label']== 'RightEye':
#             classes[1] = 1
#             coords[2] = np.squeeze(label['shapes'][0]['points'])[0]
#             coords[3] = np.squeeze(label['shapes'][0]['points'])[1]

        
#         np.divide(coords, [640,480,640,480])
            
#     try: 
#         for x in range(120):
#             keypoints = [(coords[:2]), (coords[2:])]
#             augmented = augmentor(image=img, keypoints=keypoints, class_labels=['LeftEye','RightEye'])
#             cv2.imwrite(os.path.join('aug_data', 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

#             annotation = {}
#             annotation['image'] = image
#             annotation['class'] = [0,0]
#             annotation['keypoints'] = [0,0,0,0]

#             if os.path.exists(label_path):
#                 if len(augmented['keypoints']) > 0: 
#                     for idx, cl in enumerate(augmented['class_labels']):
#                         if cl == 'LeftEye': 
#                             annotation['class'][0] = 1 
#                             annotation['keypoints'][0] = augmented['keypoints'][idx][0]
#                             annotation['keypoints'][1] = augmented['keypoints'][idx][1]
#                         if cl == 'RightEye': 
#                             annotation['class'][1] = 1 
#                             annotation['keypoints'][2] = augmented['keypoints'][idx][0]
#                             annotation['keypoints'][3] = augmented['keypoints'][idx][1]
                            
#             annotation['keypoints'] = list(np.divide(annotation['keypoints'], [450,450,450,450]))


#             #with open(os.path.join('aug_data', 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
#             #    json.dump(annotation, f)

#     except Exception as e:
#         print(e)